# Pipeline 3: A RAG Pinecone vector store for Generative AI

copyright 2024, Denis Rothman



#Installing the environment

In [1]:
!pip install openai==1.40.3
!pip install pinecone-client==5.0.1

In [2]:
f = open("pinecone.txt", "r")
PINECONE_API_KEY=f.readline()
f.close()

In [3]:
f = open("api_key.txt", "r")
API_KEY=f.readline()
f.close()

In [4]:
#The OpenAI Key
import os
import openai
os.environ['OPENAI_API_KEY'] =API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

#  The Pinecone index

In [5]:
import os
from pinecone import Pinecone, ServerlessSpec

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'

from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY)

/home/dk/miniconda3/envs/rag1/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
from pinecone import ServerlessSpec

index_name = 'bank-index-50000'
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [7]:
import time
import pinecone
# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of text-embedding-ada-002
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    time.sleep(1)

# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 50000}},
 'total_vector_count': 50000}

# RAG with GPT-4o

# Query the dataset

In [8]:
import openai
import time

embedding_model = "text-embedding-3-small"

# Initialize the OpenAI client
client = openai.OpenAI()

def get_embedding(text, model=embedding_model):
    text = text.replace("\n", " ")
    response = client.embeddings.create(input=[text], model=model)
    embedding = response.data[0].embedding
    return embedding

## Querying a target vector

In [9]:
import time
start_time = time.time()  # Start timing before the request
# Target vector
query_text = "Customer Henderson CreditScore 599 Age 37Tenure 2Balance 0.0NumOfProducts 1HasCrCard 1IsActiveMember 1EstimatedSalary 107000.88Exited 1Complain 1Satisfaction Score 2Card Type DIAMONDPoint Earned 501"
query_embedding = get_embedding(query_text,model=embedding_model)

In [10]:
# Perform the query using the embedding
query_results = index.query(
    vector=query_embedding,
    include_metadata=True,
    top_k=1
)
# Print the query results along with metadata
print("Query Results:")
for match in query_results['matches']:
    print(f"ID: {match['id']}, Score: {match['score']}")
    if 'metadata' in match and 'text' in match['metadata']:
        print(f"Text: {match['metadata']['text']}")
    else:
        print("No metadata available.")

response_time = time.time() - start_time              # Measure response time
print(f"Querying response time: {response_time:.2f} seconds")  # Print response time

Query Results:
ID: 1686, Score: 0.85504812
Text: CustomerId: 15648064 CreditScore: 649 Age: 33 Tenure: 2 Balance: 0.0 NumOfProducts: 2 HasCrCard: 1 IsActiveMember: 0 EstimatedSalary: 2010.98 Exited: 0 Complain: 0 Satisfaction Score: 3 Card Type: DIAMOND Point Earned: 720
Querying response time: 2.38 seconds


## Extract Relevant Texts

In [11]:
relevant_texts = [match['metadata']['text'] for match in query_results['matches'] if 'metadata' in match and 'text' in match['metadata']]

# Join all items in the list into a single string separated by a specific delimiter (e.g., a newline or space)
combined_text = '\n'.join(relevant_texts)  # Using newline as a separator for readability
print(combined_text)

CustomerId: 15648064 CreditScore: 649 Age: 33 Tenure: 2 Balance: 0.0 NumOfProducts: 2 HasCrCard: 1 IsActiveMember: 0 EstimatedSalary: 2010.98 Exited: 0 Complain: 0 Satisfaction Score: 3 Card Type: DIAMOND Point Earned: 720


## Augmented prompt

In [12]:
# Combine texts into a single string, separated by new lines
combined_context = "\n".join(relevant_texts)
#prompt
query_prompt="I have this customer bank record with interesting information on age, credit score and more and similar customers. What could I suggest to keep them in my bank in an email with an url to get new advantages based on the fields for each Customer ID:"
itext=query_prompt+ query_text+combined_context
# Augmented input
print("Prompt for the Generative AI model:", itext)

Prompt for the Generative AI model: I have this customer bank record with interesting information on age, credit score and more and similar customers. What could I suggest to keep them in my bank in an email with an url to get new advantages based on the fields for each Customer ID:Customer Henderson CreditScore 599 Age 37Tenure 2Balance 0.0NumOfProducts 1HasCrCard 1IsActiveMember 1EstimatedSalary 107000.88Exited 1Complain 1Satisfaction Score 2Card Type DIAMONDPoint Earned 501CustomerId: 15648064 CreditScore: 649 Age: 33 Tenure: 2 Balance: 0.0 NumOfProducts: 2 HasCrCard: 1 IsActiveMember: 0 EstimatedSalary: 2010.98 Exited: 0 Complain: 0 Satisfaction Score: 3 Card Type: DIAMOND Point Earned: 720


## Augmented generation

In [13]:
import openai

gpt_model = "gpt-4o"

import time
start_time = time.time()  # Start timing before the request

response = client.chat.completions.create(
  model=gpt_model,
  messages=[
    {
      "role": "system",
      "content": "You are the community manager can write engaging email based on the text you have. Do not use a surname but simply Dear Valued Customer instead."
    },
    {
      "role": "user",
      "content": itext
    }
  ],
  temperature=0,
  max_tokens=300,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

response_time = time.time() - start_time              # Measure response time
print(f"Querying response time: {response_time:.2f} seconds")  # Print response time

Subject: Unlock Exclusive Benefits with Our New Customer Loyalty Program!

Dear Valued Customer,

We hope this message finds you well. At [Bank Name], we are constantly striving to enhance your banking experience and provide you with the best services tailored to your needs. We noticed that you are a valued member of our community, and we want to ensure you are getting the most out of your relationship with us.

We are excited to introduce our new Customer Loyalty Program, designed to reward you with exclusive benefits and offers. As a DIAMOND cardholder, you are already enjoying premium services, but there's so much more waiting for you!

**Here's what you can look forward to:**

- **Personalized Financial Advice:** Our experts are ready to help you improve your credit score and manage your finances more effectively.
- **Exclusive Offers:** Enjoy special discounts and promotions tailored just for you.
- **Enhanced Rewards:** Earn more points with every transaction and redeem them for 